In [82]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [83]:
import logging
logging.basicConfig(level=logging.INFO)

In [84]:
def sentence_former(lst,sent_model):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    text_join = " [MASK] ".join(lst)
    text = '[CLS] {} [SEP]'.format(text_join)
    print(text)
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    # Create the segments tensors.
    segments_ids = [0] * len(tokenized_text)

    correct_text = []

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # sent_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
    # sent_model.eval()
    masked_index = [i for i, x in enumerate(tokenized_text) if x == '[MASK]']
    # Predict all tokens
    with torch.no_grad():
        predictions = sent_model(tokens_tensor, segments_tensors)
    for i in masked_index:
        #masked_index = tokenized_text.index(word)
        predicted_index = torch.argmax(predictions[0, i]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
        correct_text.append(predicted_token)
        
    return correct_text , text
